In [179]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
import requests
from selenium import webdriver

In [205]:
chrome_driver_path = '/Users/jackgoettle/Downloads/chromedriver'
browser = webdriver.Chrome(executable_path = chrome_driver_path)
base_url = 'https://rotogrinders.com/projected-stats/nba?site=fanduel&date='

In [206]:
szn_start_2016 = datetime.datetime(2016, 10, 25)
szn_start_2017 = datetime.datetime(2017, 10, 17)
szn_start_2018 = datetime.datetime(2018, 10, 16)
szn_start_2019 = datetime.datetime(2019, 10, 22)

szn_end_2016 = datetime.datetime(2017, 4, 12)
szn_end_2017 = datetime.datetime(2018, 4, 11)
szn_end_2018 = datetime.datetime(2019, 4, 10)
szn_end_2019 = datetime.datetime(2020, 3, 11)

In [207]:
# convert attrbs to int
def to_int(num) :
    try :
        return int(num)
    except :
        return num

# convert attrbs to float
def to_float(num) :
    try :
        return float(num)
    except :
        return num
def format_date(y, m, d) :
    date = str(y) + '-'
    if m < 10 :
        date = date + '0' + str(m) + '-'
    else :
        date = date + str(m) + '-'
    if d < 10 :
        date = date + '0' + str(d)
    else :
        date = date + str(d)
    return date

In [208]:
def scrape(y, m, d) :
    date = format_date(y, m, d)
    url = base_url + date
    print(url)
    browser.get(url)
    browser.implicitly_wait(10)
    table = None
    try :
        table = browser.find_element_by_class_name('rgtable')
    except :
        print('Err: ' + date)
        return None
    columns = browser.find_elements_by_class_name('rgt-col')
    count = 0
    num_rows = 0
    col_arr = []
    ignore_cols = [1, 2, 4, 9, 10, 17, 18, 19, 21, 22]
    for col in columns :
        text = col.text.split('\n')
        # get num rows bc name column can't have any
        if count in ignore_cols :
            count += 1
            continue
        if count == 0 :
            num_rows = len(text)
        # if no null values, just append column
        if len(text) == num_rows :
            col_arr.append(text)
        # must include null values. yuck!
        else :
            vals = []
            first = True
            c = 0
            for entry in text :
                if first or ' ' not in entry:
                    first = False
                    vals.append(entry)
                else :
                    entry = entry.replace('%', '')
                    for idx in range(0, len(entry)):
                        if entry[idx] == ' ' :
                            vals.append(None)
                        else :
                            vals.append(to_int(entry[idx:]))
                            break
            col_arr.append(vals)
        count += 1
    
    df = pd.DataFrame(col_arr)
    df = df.T
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    return df

In [211]:
date = szn_start_2019
count = 0
dfs_2017 = None
while date <= szn_end_2019 :
    df = scrape(date.year, date.month, date.day)
    if df is None :
        date = date + timedelta(days = 1)
        continue
    df['GAME DATE'] = date
    if dfs_2017 is None :
        dfs_2017 = df
    else :
        dfs_2017 = dfs_2017.append(df)
    date = date + timedelta(days = 1)

https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-10-22
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-10-23
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-10-24
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-10-25
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-10-26
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-10-27
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-10-28
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-10-29
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-10-30
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-10-31
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-11-01
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-11-02
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2019-11-03
https://rotogrinders.com/projected-sta

https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-10
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-11
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-12
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-13
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-14
Err: 2020-02-14
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-15
Err: 2020-02-15
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-16
Err: 2020-02-16
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-17
Err: 2020-02-17
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-18
Err: 2020-02-18
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-19
Err: 2020-02-19
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-20
https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-21
https://rotogrin

In [210]:
scrape(2020, 2, 3)

https://rotogrinders.com/projected-stats/nba?site=fanduel&date=2020-02-03


,Name,Position,Rank,Average,pOWN%,ContR,DvP,DvPRank,O/U,Line,Total,Movement,PMIN,Points,Pt/$/K
1,Bruce Brown,SG,None,None,21.00%,1.318,20.41,23,223.5,380,107,0.25,32.00,27.68,6.92
2,Damion Lee,SG,None,None,29.00%,1.057,46.07,20,233.5,210,113.5,-1,31.00,30.65,6.81
3,Alec Burks,SF,None,None,44.00%,0.772,26.80,20,233.5,210,113.5,-1,31.00,33.98,6.66
4,Damian Jones,C,None,None,3.00%,8.690,26.60,7,231,168,113,1.5,30.00,26.07,6.36
5,Reggie Jackson,PG,None,None,23.00%,1.233,44.89,16,223.5,380,107,0.25,30.00,28.35,6.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,Jordan Bell,PF,None,None,0.10%,None,29.94,17,226,102,112.25,-0.5,4.00,3.84,1.1
198,Carsen Edwards,PG,None,None,0.10%,None,48.40,26,231,-200,118,1,5.00,3.63,1.04
199,Goga Bitadze,C,None,None,0.10%,None,28.05,5,213.5,-235,109.5,0.25,4.00,3.41,0.97
200,Tim Frazier,PG,None,None,0.10%,None,20.41,23,223.5,380,107,0.25,4.00,3.26,0.91


In [212]:
dfs_2017 = dfs_2017.reset_index(drop = True)

In [216]:
dfs_2017.to_csv('dfs_advanced_2019.csv', index = False)

In [214]:
dfs_2017.isna().sum()

0
Name             0
Position         0
Rank         16937
Average      16937
pOWN%          185
ContR         8622
DvP              7
DvPRank          7
O/U              0
Line             0
Total            0
Movement      2730
PMIN             0
Points           0
Pt/$/K           0
GAME DATE        0
dtype: int64